In [ ]:
//Oscar Delgado and Colins LISP projects 
//Souce: https://github.com/jaratec/spl/tree/master/src/main/scala/net/jaratec/spl

In [ ]:
//Imports
//import $ivy.`org.scala-lang.modules::scala-parser-combinators:1.1.2`
import scala.util.parsing.combinator._
import scala.util.control.Exception.allCatch

In [ ]:
/*

Check List:
Type:
Booleans T - Defines
Numbers T scala.Int
Characters T - scala.Char
Strings T = String
Symbols T
Pairs and Lists T
Vectors F -
Procedures F -

*/

In [35]:
// TYPES

sealed trait Bool
case object True extends Bool
case object False extends Bool

sealed trait List[+A]
case object Empty extends List[Nothing]
case class Cons[A](x : A, xs : List[A]) extends List[A]

sealed trait Pair[+A,+B]
case class MkPair[A,B](fst : A, snd : B) extends Pair[A,B]

sealed trait Symbols
case object EmptyS extends Symbols 
case class Extend(k : Expr, v : Expr, env : Symbols) extends Symbols
//case class Procedure(f: String, e: Expr, sigma: Symbols ) extends Symbols


sealed trait Vectors
case object EmptyV extends Vectors
case class VecAppend(p : Int, v : Expr, env : Vectors) extends Vectors



//sealed trait Procedures

sealed trait Expr
case class LispNum(x : scala.Int) extends Expr
case class LispBin(x : Boolean) extends Expr
case class Ident(x : String) extends Expr
case class Plus(x : Expr, y : Expr) extends Expr
case class Minus(x : Expr, y : Expr) extends Expr
case class Mult(x : Expr, y : Expr) extends Expr
case class Div(x : Expr, y : Expr) extends Expr
//case class **(x : Expr, y : Expr) extends Expr
case class Neg(x : Expr) extends Expr
case class Eq(x : Expr, y : Expr) extends Expr
case class And(x : Expr, y : Expr) extends Expr
case class Or(x : Expr, y : Expr) extends Expr
case class IfThenElse(p : Expr, t : Expr, f : Expr) extends Expr
case class Let(id : Expr, x : Expr, y : Expr) extends Expr
case class FunDef(id: Expr, x : Expr) extends Expr
case class FunCall(id : Expr, x : Expr) extends Expr
//case class LetRec (id1 : String, id2 : String, x : Expr, y : Expr) extends Expr
case class Procedure(arg : Expr, e : Expr, pi : Symbols) extends Expr
case object Error extends Expr
case object OpenP extends Expr
case object CloseP extends Expr
case object holder extends Expr
case class tuple (elem : List[Expr]) extends Expr

holder

In [37]:
sealed trait Maybe[+A]
case object Nothing extends Maybe[Nothing]
case class Just[A](fromJust : A) extends Maybe[A]

def lookup(dict : Symbols, k : Expr) : Maybe[Expr] = dict match {
    case EmptyS        => Nothing
    case Extend(k1, v, xs) => k == k1 match {
        case true  => Just(v)
        case false => lookup(xs, k)
    }
}

//Definition
var GStore : Symbols = EmptyS
GStore = Extend(Ident("x") , LispNum(10),GStore)
lookup(GStore,Ident("lool"))

Nothing

In [38]:
def bin_Arith(t : Char, v1 : Expr, v2 : Expr) : Expr = (t,v1,v2) match{
    case ('+',LispNum(v1),LispNum(v2)) => LispNum(v1+v2) 
    case ('-',LispNum(v1),LispNum(v2)) => LispNum(v1-v2) 
    case ('*',LispNum(v1),LispNum(v2)) => LispNum(v1*v2) 
    case ('/',LispNum(v1),LispNum(v2)) => LispNum(v1/v2) 
    case _ => Error
}
def bool_Arith(b : Char, v1 : Expr, v2 : Expr) : Expr = (b,v1,v2) match{
    case ('&',LispBin(v1),LispBin(v2)) => LispBin(v1 && v2) 
    case ('|',LispBin(v1),LispBin(v2)) => LispBin(v1 || v2) 
    case _ => Error
}


def Eval(store : Symbols, expr : Expr) : Expr = expr match{
    case LispNum(n) => LispNum(n)
    case LispBin(n) => LispBin(n)
    case Ident(x) => lookup(store , Ident(x)) match{
        case Just(v) => v
        case _ => Error
    }
    case Plus(x,y) => bin_Arith('+' , Eval(store,x),Eval(store,y))
    case Minus(x,y) => bin_Arith('-' , Eval(store,x),Eval(store,y))
    case Mult(x,y) => bin_Arith('*' , Eval(store,x),Eval(store,y))
    case Div(x,y) => bin_Arith('/' , Eval(store,x),Eval(store,y))
    case Eq(x,y) => Eval(store,x) == Eval(store,y) match{
        case x => LispBin(x)
    }
    case And(x,y) => bool_Arith('&' , Eval(store,x),Eval(store,y))
    case Or(x,y) => bool_Arith('|' , Eval(store,x),Eval(store,y))
    case Neg(x) => Eval(store,x) match{
        case LispNum(x) => LispNum(-x)
        case LispBin(x) => LispBin(!x)
        case _ => Error
    }  
    case IfThenElse(p, e_t, e_f) => Eval(store, p) match{
        case LispBin(true)  => Eval(store, e_t)
        case LispBin(false) => Eval(store, e_f)
        case _             => Error
    }
    case Let(x,y,z) => Eval(Extend(x,Eval(store,y),store),z)
    case FunDef(x , e1) => Procedure(x,e1,store)
    case FunCall(e1 , e2) => Eval(store,e1) match{
        case Procedure(p,e3,pi)=> Eval(store,e2) match{
            case x => {
            val new_env = Extend(p, x, pi)
            Eval(new_env , e3)
            }
        }
        case _ => Error
    }
    case Procedure(x,y,z) => Procedure(x,y,z)
    case Error=>Error
    
}

<console>:149: warning: match may not be exhaustive.
It would fail on the following inputs: CloseP, OpenP, holder
       def Eval(store : Symbols, expr : Expr) : Expr = expr match{
                                                       ^
bin_Arith: (t: Char, v1: Expr, v2: Expr)Expr
bool_Arith: (b: Char, v1: Expr, v2: Expr)Expr
Eval: (store: Symbols, expr: Expr)Expr


In [ ]:
var testSArray : Array[String] = Array[String]("(", "+","(", "2", "3", ")", ")")
def ItoS(input : String) : Array[String] = input match{
    case input => input.split(" ")
}

def isNumber(s: String): Boolean = (allCatch opt s.toDouble).isDefined

var i = 0;

def makeArrayE(stringA : Array[String]) : Array[Expr] = {
    var exprA : Array[Expr] = new Array[Expr](stringA.length)
    while(i < stringA.length){
        if(stringA.apply(i) == "("){
            exprA(i) = OpenP;

        }else if(stringA.apply(i) == ")"){
            exprA(i) = CloseP;

        }else if(stringA.apply(i) == "+"){
            exprA(i) = Plus(holder,holder);

        }else if(stringA.apply(i) == "-"){
            exprA(i) = Minus(holder,holder);

        }else if(stringA.apply(i) == "*"){
            exprA(i) = Mult(holder,holder);

        }else if(stringA.apply(i) == "/"){
            exprA(i) = Div(holder,holder);

        }else if(stringA.apply(i) == "="){
            exprA(i) = Eq(holder,holder);

        }else if(stringA.apply(i) == "true"){
            exprA(i) = LispBin(true);

        }else if(stringA.apply(i) == "false"){
            exprA(i) = LispBin(false);

        }else if(isNumber(stringA.apply(i)) == true){
            exprA(i) = LispNum(stringA.apply(i).toInt);

        }else if(stringA.apply(i) == "&&"){
            exprA(i) = And(holder,holder); 

        }else if(stringA.apply(i) == "||"){
            exprA(i) = Or(holder,holder);

        }else if(stringA.apply(i) == "!"){
            exprA(i) = Neg(holder);

        }else if(stringA.apply(i) == "x"){
            exprA(i) = Ident(stringA.apply(i));

        }else if(stringA.apply(i) == "y"){
            exprA(i) = Ident(stringA.apply(i));

        }else if(stringA.apply(i) == "z"){
            exprA(i) = Ident(stringA.apply(i));

        }else if(stringA.apply(i) == "if"){
            exprA(i) = IfThenElse(holder,holder,holder);

        //}else if(stringA.apply(i) == "define"){
        //   exprA(i) = Procedure(Empty,holder,holder);

        }else{
             exprA(i) = Error;
        }
        i += 1;
    }
    return exprA
}

makeArrayE(ItoS("( + ( - 2 || 3 4 ) 3 )"))
//while(i < )a1


In [4]:
val a1 = Array(OpenP, Plus, OpenP, Plus, LispNum(2), LispNum(3), CloseP, LispNum(1), CloseP)
val a2 : Array[Expr] = Array(OpenP,Plus(holder,holder),LispNum(2),LispNum(3),CloseP)
val a3 : Array[Expr] = Array(OpenP,LispNum(10),CloseP)

def AtoE(A : Array[Expr]) : Expr = A {
    var i : Int = 0;
    var lol : Expr = Error
    while (i < A.length){
        if (A.apply(i) == OpenP ){
            lol = A.apply(i)
        }
        if (A.apply(i) == Plus(holder,holder))
        if (A.apply(i) == Minus(holder,holder))
        if (A.apply(i) == Mult(holder,holder))
        if (A.apply(i) == Div(holder,holder))
        if (A.apply(i) == Plus(holder,holder))
        if (A.apply(i) == Eq(holder,holder))
        if (A.apply(i) == And(holder,holder))
        if (A.apply(i) == Or(holder,holder))
        if (A.apply(i) == IfThenElse(holder,holder,holder))
        if (A.apply(i) == Let(holder,holder,holder))
        if (A.apply(i) == FunDef(holder,holder))
        if (A.apply(i) == FunCall(holder,holder))
        if (A.apply(i) == Procedure(holder,holder,holder))
    }
    return lol
}
//a2
//a3


LispNum(10)

In [ ]:
var inhole = AtoE(a3)
//Eval(GStore , inhole)

<console>: 20

LispNum(10)